## Content Based Recommendation On Movies

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rake_nltk import Rake
import pandas as pd
import numpy as np

D:\conda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\conda\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\conda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fany\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
pd.set_option('display.max_columns',100)
data=pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
data.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [5]:
data=data[['Title','Genre','Director','Actors','Plot']]
data.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [6]:
data['Actors']=data['Actors'].apply(lambda x:x.lower().split(',')[:3])
data.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"[tim robbins, morgan freeman, bob gunton]",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"[marlon brando, al pacino, james caan]",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"[al pacino, robert duvall, diane keaton]",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"[christian bale, heath ledger, aaron eckhart]",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"[martin balsam, john fiedler, lee j. cobb]",A jury holdout attempts to prevent a miscarria...


In [7]:
data['Director']=data['Director'].apply(lambda x: x.lower().replace(' ',''))
data.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",frankdarabont,"[tim robbins, morgan freeman, bob gunton]",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",francisfordcoppola,"[marlon brando, al pacino, james caan]",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",francisfordcoppola,"[al pacino, robert duvall, diane keaton]",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",christophernolan,"[christian bale, heath ledger, aaron eckhart]",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",sidneylumet,"[martin balsam, john fiedler, lee j. cobb]",A jury holdout attempts to prevent a miscarria...


In [8]:
data['Genre']=data['Genre'].map(lambda x: x.lower().split(','))
data.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"[crime, drama]",frankdarabont,"[tim robbins, morgan freeman, bob gunton]",Two imprisoned men bond over a number of years...
1,The Godfather,"[crime, drama]",francisfordcoppola,"[marlon brando, al pacino, james caan]",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[al pacino, robert duvall, diane keaton]",The early life and career of Vito Corleone in ...
3,The Dark Knight,"[action, crime, drama]",christophernolan,"[christian bale, heath ledger, aaron eckhart]",When the menace known as the Joker emerges fro...
4,12 Angry Men,"[crime, drama]",sidneylumet,"[martin balsam, john fiedler, lee j. cobb]",A jury holdout attempts to prevent a miscarria...


In [9]:
data['Plot']=data['Plot'].apply(lambda x:x.lower())
data.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"[crime, drama]",frankdarabont,"[tim robbins, morgan freeman, bob gunton]",two imprisoned men bond over a number of years...
1,The Godfather,"[crime, drama]",francisfordcoppola,"[marlon brando, al pacino, james caan]",the aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[al pacino, robert duvall, diane keaton]",the early life and career of vito corleone in ...
3,The Dark Knight,"[action, crime, drama]",christophernolan,"[christian bale, heath ledger, aaron eckhart]",when the menace known as the joker emerges fro...
4,12 Angry Men,"[crime, drama]",sidneylumet,"[martin balsam, john fiedler, lee j. cobb]",a jury holdout attempts to prevent a miscarria...


In [10]:
data['extracted_Plot']=''
for index,row in data.iterrows():
    plot=row['Plot']
    r=Rake()
    r.extract_keywords_from_text(plot) # extract_keywords_from_text() extracts the key words from the plot
    word_scores_dict=r.get_word_degrees() #get_word_degrees() gives the dictionary of the word and its score 
    row['extracted_Plot']=list(word_scores_dict.keys())
data.head()

,Title,Genre,Director,Actors,Plot,extracted_Plot
0,The Shawshank Redemption,"[crime, drama]",frankdarabont,"[tim robbins, morgan freeman, bob gunton]",two imprisoned men bond over a number of years...,"[two, imprisoned, men, bond, number, years, fi..."
1,The Godfather,"[crime, drama]",francisfordcoppola,"[marlon brando, al pacino, james caan]",the aging patriarch of an organized crime dyna...,"[aging, patriarch, organized, crime, dynasty, ..."
2,The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[al pacino, robert duvall, diane keaton]",the early life and career of vito corleone in ...,"[early, life, career, vito, corleone, 1920s, n..."
3,The Dark Knight,"[action, crime, drama]",christophernolan,"[christian bale, heath ledger, aaron eckhart]",when the menace known as the joker emerges fro...,"[menace, known, joker, emerges, mysterious, pa..."
4,12 Angry Men,"[crime, drama]",sidneylumet,"[martin balsam, john fiedler, lee j. cobb]",a jury holdout attempts to prevent a miscarria...,"[jury, holdout, attempts, prevent, miscarriage..."


In [11]:
data[['Plot','extracted_Plot']].head()

,Plot,extracted_Plot
0,two imprisoned men bond over a number of years...,"[two, imprisoned, men, bond, number, years, fi..."
1,the aging patriarch of an organized crime dyna...,"[aging, patriarch, organized, crime, dynasty, ..."
2,the early life and career of vito corleone in ...,"[early, life, career, vito, corleone, 1920s, n..."
3,when the menace known as the joker emerges fro...,"[menace, known, joker, emerges, mysterious, pa..."
4,a jury holdout attempts to prevent a miscarria...,"[jury, holdout, attempts, prevent, miscarriage..."


In [12]:
data=data.drop('Plot',axis=1)
data.head()

,Title,Genre,Director,Actors,extracted_Plot
0,The Shawshank Redemption,"[crime, drama]",frankdarabont,"[tim robbins, morgan freeman, bob gunton]","[two, imprisoned, men, bond, number, years, fi..."
1,The Godfather,"[crime, drama]",francisfordcoppola,"[marlon brando, al pacino, james caan]","[aging, patriarch, organized, crime, dynasty, ..."
2,The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[al pacino, robert duvall, diane keaton]","[early, life, career, vito, corleone, 1920s, n..."
3,The Dark Knight,"[action, crime, drama]",christophernolan,"[christian bale, heath ledger, aaron eckhart]","[menace, known, joker, emerges, mysterious, pa..."
4,12 Angry Men,"[crime, drama]",sidneylumet,"[martin balsam, john fiedler, lee j. cobb]","[jury, holdout, attempts, prevent, miscarriage..."


In [13]:
data.set_index('Title',inplace=True)
data.head()

,Genre,Director,Actors,extracted_Plot
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[tim robbins, morgan freeman, bob gunton]","[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlon brando, al pacino, james caan]","[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[al pacino, robert duvall, diane keaton]","[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christian bale, heath ledger, aaron eckhart]","[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martin balsam, john fiedler, lee j. cobb]","[jury, holdout, attempts, prevent, miscarriage..."


In [14]:
data['bag_of_words']=''
columns=data.columns
for index,row in data.iterrows():
    word=''
    for col in columns:
        if col!='Director':
            word=word+' '.join(row[col])+' '
        else:    
            word=word+row[col]+' '
    row['bag_of_words']=word
data=data.drop([col for col in data.columns if col!='bag_of_words'],axis=1)
data.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont tim robbins morgan...
The Godfather,crime drama francisfordcoppola marlon brando ...
The Godfather: Part II,crime drama francisfordcoppola al pacino rob...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martin balsam john f...


In [15]:
count=CountVectorizer()
vectorMatrix=count.fit_transform(data['bag_of_words'])
indeces=pd.Series(data.index)
indeces[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [16]:
vectorMatrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
print(vectorMatrix[0,:])

  (0, 662)	1
  (0, 863)	1
  (0, 1133)	1
  (0, 2905)	1
  (0, 2462)	1
  (0, 1980)	1
  (0, 1146)	1
  (0, 339)	1
  (0, 1293)	1
  (0, 2988)	1
  (0, 1452)	1
  (0, 1904)	1
  (0, 347)	1
  (0, 2082)	1
  (0, 3193)	1
  (0, 1079)	1
  (0, 2691)	1
  (0, 991)	1
  (0, 2372)	1
  (0, 61)	1
  (0, 588)	1
  (0, 744)	1


In [18]:
count.vocabulary_

{'crime': 662,
 'drama': 863,
 'frankdarabont': 1133,
 'tim': 2905,
 'robbins': 2462,
 'morgan': 1980,
 'freeman': 1146,
 'bob': 339,
 'gunton': 1293,
 'two': 2988,
 'imprisoned': 1452,
 'men': 1904,
 'bond': 347,
 'number': 2082,
 'years': 3193,
 'finding': 1079,
 'solace': 2691,
 'eventual': 991,
 'redemption': 2372,
 'acts': 61,
 'common': 588,
 'decency': 744,
 'francisfordcoppola': 1130,
 'marlon': 1839,
 'brando': 381,
 'al': 93,
 'pacino': 2141,
 'james': 1534,
 'caan': 434,
 'aging': 86,
 'patriarch': 2170,
 'organized': 2119,
 'dynasty': 893,
 'transfers': 2952,
 'control': 626,
 'clandestine': 538,
 'empire': 947,
 'reluctant': 2394,
 'son': 2702,
 'robert': 2464,
 'duvall': 890,
 'diane': 802,
 'keaton': 1623,
 'early': 895,
 'life': 1731,
 'career': 454,
 'vito': 3078,
 'corleone': 638,
 '1920s': 8,
 'new': 2047,
 'york': 3197,
 'portrayed': 2252,
 'michael': 1922,
 'expands': 1012,
 'tightens': 2904,
 'grip': 1277,
 'family': 1038,
 'syndicate': 2833,
 'action': 58,
 'chri

In [19]:
similarMovies=cosine_similarity(vectorMatrix,vectorMatrix)
similarMovies

array([[1.        , 0.13636364, 0.12087344, ..., 0.04545455, 0.04545455,
        0.04767313],
       [0.13636364, 1.        , 0.36262033, ..., 0.04545455, 0.04545455,
        0.04767313],
       [0.12087344, 0.36262033, 1.        , ..., 0.04029115, 0.04029115,
        0.04225771],
       ...,
       [0.04545455, 0.04545455, 0.04029115, ..., 1.        , 0.04545455,
        0.04767313],
       [0.04545455, 0.04545455, 0.04029115, ..., 0.04545455, 1.        ,
        0.04767313],
       [0.04767313, 0.04767313, 0.04225771, ..., 0.04767313, 0.04767313,
        1.        ]])

In [20]:
def MovieSuggetions(movie,similarMovies=similarMovies):
    suggestedMoviesList=[]
    idx=indeces[indeces==movie].index[0]
    titleList=pd.Series(similarMovies[idx]).sort_values(ascending=False)
    top_10_Movies=list(titleList[:11].index)
    print(top_10_Movies)
    for i in top_10_Movies:
        suggestedMoviesList.append(list(data.index)[i])
    print(suggestedMoviesList)

In [21]:
MovieSuggetions('Rush')

[142, 111, 179, 141, 60, 3, 151, 200, 228, 32, 33]
['Rush', 'A Beautiful Mind', 'The Man Who Shot Liberty Valance', 'In the Name of the Father', 'Braveheart', 'The Dark Knight', 'Butch Cassidy and the Sundance Kid', 'Dog Day Afternoon', 'Pink Floyd: The Wall', 'The Pianist', 'Rear Window']
